# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [11]:
def make_completion_rows(transcript=transcript, portfolio=portfolio):
    
    for customer in transcript['person'].unique():

        everything = transcript[transcript['person'] == customer]
        received = everything[everything['event'] == 'offer received']
    
        info_trans = []
        for iloc in received.index:
            offer = received.loc[iloc]['value']
            offer_id = offer['offer id']
            offer_row = portfolio[portfolio['id'] == offer_id]
        
            if offer_row.iloc[0]['offer_type'] == 'informational':
                offer_length = offer_row['duration']
                offer_hours = 24*offer_length
                offer_start = received.loc[iloc]['time']
                offer_end= offer_start+ offer_hours
                when = everything['time']
                influenced = everything[everything['event']=='transaction']['time'].where(everything['time'].isin(np.arange(offer_start, offer_end))).dropna().index
                influenced_dict = {offer_id: influenced}
                info_trans.append(influenced_dict)
            else:
                pass  
        
        if len(info_trans) >0:
            info_dict = info_trans[0]
            offer_id = list(info_trans[0])[0]#info_trans[0].keys().tolist()
            index = info_dict[offer_id]
            #index[0]
        
            if len(index) >0:
                new_idx = float(index[0]) +.1

                transcript.loc[new_idx] = ['offer completed', transcript.loc[index[0]]['person'], transcript.loc[index[0]]['time'], [{'offer id' : offer_id, 'reward': 0}]]
            else:
                pass
        else:
            pass
    
    
    return transcript.sort_index()
    
added_transcript = make_completion_rows()  


In [12]:
new_columns = pd.DataFrame(added_transcript['value'].apply(pd.Series))
new_columns['offer id'].fillna(new_columns['offer_id'], inplace = True)
expanded_transcript = pd.concat([added_transcript, new_columns], axis =1)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '>' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '>' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [10]:
new_columns = pd.DataFrame(added_transcript['value'].apply(pd.Series))
new_columns

,0
0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
5,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
6,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
7,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
8,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
9,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}


In [8]:
new_columns[0].apply(pd.Series)

,0
0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
5,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
6,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
7,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
8,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
9,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}


In [22]:
def create_unoffered_dict(transcript=expanded_transcript):

#Find transactions with offer completions at the same time
    
    unoffered_dict = {}
    for customer in transcript['person'].unique():
        unoffered_list = []
        offered_list = []
        everything = transcript[transcript['person'] == customer]
        transactions = everything[everything['event'] == 'transaction']
        
        transaction_times = transactions['time']
        completions = everything[everything['event'] == 'offer completed']
        completion_times = completions['time']
        for transaction in transactions.iterrows():
            transaction_time = transaction[1]['time']
            
            if transaction_time in completion_times.values:
                offered_list.append(transcript.loc[transaction[0]])
            else:
                unoffered_list.append(transcript.loc[transaction[0]]['amount'])
        unoffered_dict[customer] = unoffered_list
    return (unoffered_dict)

unoffered_dict = create_unoffered_dict()

In [6]:
added_transcript.to_csv('added_transcript_save', index = False)

In [24]:
np.save('uo_dict.npy', unoffered_dict)

In [2]:
unoffered_dict = np.load('uo_dict.npy').item()

In [9]:
added_transcript = pd.read_csv('added_transcript_save')

In [13]:
def create_timings(expanded_transcript=expanded_transcript):


    timings = pd.pivot_table(expanded_transcript, values = 'time', index = ['person', 'offer id'], columns = 'event', aggfunc = pd.Series.tolist)
    return (timings, expanded_transcript)

timings, expanded_transcript = create_timings()
timings

event                                                              offer completed  \
person                           offer id                                            
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5            [576]   
                                 3f207df678b143eea3cee63160fa8bed              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [414]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [528]   
00116118485d4dfda04fdbaba9a87b5c f19421c1d4aa40978ebb69ca19b0e20d              NaN   
0011e0d4e6b944f998e987f904e8c1e5 0b1e1539f2cc45b7b9fa7c272da2e1d7            [576]   
                                 2298d6c36e964ae4a3e7e9706d1fb8c2            [252]   
                                 3f207df678b143eea3cee63160fa8bed              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [576]   
0020c2b971eb4e9188eac86d93036a77 4d5c57ea9a6940dd891ad53e9dbe8da0            [510]   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 ae264e3637204a6fb9bb56bc8210ddfd              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4        [54, 510]   
0020ccbbb6d84e358d3414a3ff76cffd 2298d6c36e964ae4a3e7e9706d1fb8c2            [222]   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [600]   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [378]   
003d66b6608740288d6cc97a6903f4f0 0b1e1539f2cc45b7b9fa7c272da2e1d7            [696]   
                                 3f207df678b143eea3cee63160fa8bed              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4       [384, 504]   
00426fe3ffde4c6b9cb9ad6d077a13ea 0b1e1539f2cc45b7b9fa7c272da2e1d7              NaN   
                                 2906b810c7d4411798c6938adc9daaa5              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4            [258]   
004b041fbfe44859945daa2c7f79ee64 3f207df678b143eea3cee63160fa8bed              NaN   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [534]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [714]   
...                                                                            ...   
ffed75d3abc64b488982f50ed12878b5 4d5c57ea9a6940dd891ad53e9dbe8da0              NaN   
                                 ae264e3637204a6fb9bb56bc8210ddfd              NaN   
ffede3b700ac41d6a266fa1ba74b4f16 0b1e1539f2cc45b7b9fa7c272da2e1d7              NaN   
                                 4d5c57ea9a6940dd891ad53e9dbe8da0              NaN   
                                 f19421c1d4aa40978ebb69ca19b0e20d              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4              NaN   
fff0f0aac6c547b9b263080f09a5586a 3f207df678b143eea3cee63160fa8bed              NaN   
                                 4d5c57ea9a6940dd891ad53e9dbe8da0              NaN   
fff29fb549084123bd046dbc5ceb4faa 4d5c57ea9a6940dd891ad53e9dbe8da0       [426, 516]   
                                 ae264e3637204a6fb9bb56bc8210ddfd       [168, 684]   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [426]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [168]   
fff3ba4757bd42088c044ca26d73817a 2906b810c7d4411798c6938adc9daaa5            [528]   
                                 5a8bc65990b

In [21]:
timings.fillna('')

event                                                              offer completed  \
person                           offer id                                            
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5            [576]   
                                 3f207df678b143eea3cee63160fa8bed                    
                                 5a8bc65990b245e5a138643cd4eb9837                    
                                 f19421c1d4aa40978ebb69ca19b0e20d            [414]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [528]   
00116118485d4dfda04fdbaba9a87b5c f19421c1d4aa40978ebb69ca19b0e20d                    
0011e0d4e6b944f998e987f904e8c1e5 0b1e1539f2cc45b7b9fa7c272da2e1d7            [576]   
                                 2298d6c36e964ae4a3e7e9706d1fb8c2            [252]   
                                 3f207df678b143eea3cee63160fa8bed                    
                                 5a8bc65990b245e5a138643cd4eb9837                    
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [576]   
0020c2b971eb4e9188eac86d93036a77 4d5c57ea9a6940dd891ad53e9dbe8da0            [510]   
                                 5a8bc65990b245e5a138643cd4eb9837                    
                                 ae264e3637204a6fb9bb56bc8210ddfd                    
                                 fafdcd668e3743c1bb461111dcafc2a4        [54, 510]   
0020ccbbb6d84e358d3414a3ff76cffd 2298d6c36e964ae4a3e7e9706d1fb8c2            [222]   
                                 5a8bc65990b245e5a138643cd4eb9837                    
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [600]   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [378]   
003d66b6608740288d6cc97a6903f4f0 0b1e1539f2cc45b7b9fa7c272da2e1d7            [696]   
                                 3f207df678b143eea3cee63160fa8bed                    
                                 5a8bc65990b245e5a138643cd4eb9837                    
                                 fafdcd668e3743c1bb461111dcafc2a4       [384, 504]   
00426fe3ffde4c6b9cb9ad6d077a13ea 0b1e1539f2cc45b7b9fa7c272da2e1d7                    
                                 2906b810c7d4411798c6938adc9daaa5                    
                                 5a8bc65990b245e5a138643cd4eb9837                    
                                 fafdcd668e3743c1bb461111dcafc2a4            [258]   
004b041fbfe44859945daa2c7f79ee64 3f207df678b143eea3cee63160fa8bed                    
                                 f19421c1d4aa40978ebb69ca19b0e20d            [534]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [714]   
...                                                                            ...   
ffed75d3abc64b488982f50ed12878b5 4d5c57ea9a6940dd891ad53e9dbe8da0                    
                                 ae264e3637204a6fb9bb56bc8210ddfd                    
ffede3b700ac41d6a266fa1ba74b4f16 0b1e1539f2cc45b7b9fa7c272da2e1d7                    
                                 4d5c57ea9a6940dd891ad53e9dbe8da0                    
                                 f19421c1d4aa40978ebb69ca19b0e20d                    
                                 fafdcd668e3743c1bb461111dcafc2a4                    
fff0f0aac6c547b9b263080f09a5586a 3f207df678b143eea3cee63160fa8bed                    
                                 4d5c57ea9a6940dd891ad53e9dbe8da0                    
fff29fb549084123bd046dbc5ceb4faa 4d5c57ea9a6940dd891ad53e9dbe8da0       [426, 516]   
                                 ae264e3637204a6fb9bb56bc8210ddfd       [168, 684]   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [426]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [168]   
fff3ba4757bd42088c044ca26d73817a 2906b810c7d4411798c6938adc9daaa5            [528]   
                                 5a8bc65990b

In [24]:
def create_amounts(expanded_transcript = expanded_transcript):
    comp_trans = expanded_transcript[(expanded_transcript['event'] == 'offer completed') | (expanded_transcript['event'] == 'transaction')]
    comp_trans['trans_amount']= comp_trans['amount'].fillna(method = 'ffill')
    #filled_comp[filled_comp['person'] == '0009655768c64bdeb2e877511632db8f']
    amounts = pd.pivot_table(comp_trans, values = 'trans_amount', index = ['person', 'offer_id'], aggfunc = pd.Series.tolist)
    return (amounts)
amounts = create_amounts()
amounts

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


trans_amount
person                           offer_id                                               
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5                [10.27]
                                 f19421c1d4aa40978ebb69ca19b0e20d                 [8.57]
                                 fafdcd668e3743c1bb461111dcafc2a4                [14.11]
0011e0d4e6b944f998e987f904e8c1e5 0b1e1539f2cc45b7b9fa7c272da2e1d7                [22.05]
                                 2298d6c36e964ae4a3e7e9706d1fb8c2                [11.93]
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9                [22.05]
0020c2b971eb4e9188eac86d93036a77 4d5c57ea9a6940dd891ad53e9dbe8da0                [17.24]
                                 fafdcd668e3743c1bb461111dcafc2a4         [17.63, 17.24]
0020ccbbb6d84e358d3414a3ff76cffd 2298d6c36e964ae4a3e7e9706d1fb8c2                [11.65]
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9                [10.32]
                                 f19421c1d4aa40978ebb69ca19b0e20d                [14.53]
003d66b6608740288d6cc97a6903f4f0 0b1e1539f2cc45b7b9fa7c272da2e1d7                 [2.25]
                                 fafdcd668e3743c1bb461111dcafc2a4           [3.66, 5.01]
00426fe3ffde4c6b9cb9ad6d077a13ea fafdcd668e3743c1bb461111dcafc2a4                [16.97]
004b041fbfe44859945daa2c7f79ee64 f19421c1d4aa40978ebb69ca19b0e20d                [27.92]
                                 fafdcd668e3743c1bb461111dcafc2a4                [19.93]
004c5799adbf42868b9cff0396190900 ae264e3637204a6fb9bb56bc8210ddfd                [25.61]
                                 f19421c1d4aa40978ebb69ca19b0e20d         [22.82, 35.91]
                                 fafdcd668e3743c1bb461111dcafc2a4         [43.21, 33.67]
005500a7188546ff8a767329a2f7c76a 9b98b8c7a33c4b65b9aebfe6a799e6d9                 [9.03]
0056df74b63b4298809f0b375a304cf4 0b1e1539f2cc45b7b9fa7c272da2e1d7                [27.59]
                                 2298d6c36e964ae4a3e7e9706d1fb8c2                [29.77]
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9                [27.86]
0069a50874d846438e58acff5e594725 2906b810c7d4411798c6938adc9daaa5                 [7.97]
00715b6e55c3431cb56ff7307eb19675 0b1e1539f2cc45b7b9fa7c272da2e1d7         [28.73, 21.59]
                                 2906b810c7d4411798c6938adc9daaa5                [21.35]
                                 4d5c57ea9a6940dd891ad53e9dbe8da0                [17.86]
                                 ae264e3637204a6fb9bb56bc8210ddfd                [27.26]
0082fd87c18f45f2be70dbcbb0fb8aad 9b98b8c7a33c4b65b9aebfe6a799e6d9          [19.24, 8.95]
00840a2ca5d2408e982d56544dc14ffd 2906b810c7d4411798c6938adc9daaa5                 [6.05]
...                                                                                  ...
ffd53e5ba61d41378b66efc00d9f9682 2298d6c36e964ae4a3e7e9706d1fb8c2                 [7.49]
                                 2906b810c7d4411798c6938adc9daaa5                [13.56]
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9          [13.53, 8.45]
                                 ae264e3637204a6fb9bb56bc8210ddfd                [10.68]
ffda4545e7084ff6aaf01304fb1c1fa6 0b1e1539f2cc45b7b9fa7c272da2e1d7                [24.08]
                                 fafdcd668e3743c1bb461111dcafc2a4                [24.08]
ffdefcac307f4ca99ac1ebd51470f106 2906b810c7d4411798c6938adc9daaa5                [10.78]
                                 4d5c57ea9a6940dd891ad53e9dbe8da0                 [12.5]
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9         [11.17, 13.43]
                                 fafdcd668e3743c1bb461111dcafc2a4                [24.41]
ffe5257abf8840b395e1ee6b29894637 0b1e1539f2cc45b7b9fa7c272da2e1d7                [34.95]
                                 2906b810c7d4411798c6938adc9daaa5                [34.95]
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9      

In [51]:
#5
master_list = pd.concat([timings, amounts], axis = 1)


In [59]:
#rewards = portfolio['reward']
#offers = portfolio['id']
rewards_dict = {}
for i in np.arange(len(portfolio)):
    rewards_dict[portfolio['id'][i]] = portfolio['reward'][i]
rewards_list = []
for id in master_list.reset_index()['level_1']:
    rewards_list.append(rewards_dict[id])
    
master_list['reward'] = rewards_list
#rewards_dict

In [60]:
master_list

offer completed  \
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5            [576]   
                                 3f207df678b143eea3cee63160fa8bed              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [414]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [528]   
00116118485d4dfda04fdbaba9a87b5c f19421c1d4aa40978ebb69ca19b0e20d              NaN   
0011e0d4e6b944f998e987f904e8c1e5 0b1e1539f2cc45b7b9fa7c272da2e1d7            [576]   
                                 2298d6c36e964ae4a3e7e9706d1fb8c2            [252]   
                                 3f207df678b143eea3cee63160fa8bed              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [576]   
0020c2b971eb4e9188eac86d93036a77 4d5c57ea9a6940dd891ad53e9dbe8da0            [510]   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 ae264e3637204a6fb9bb56bc8210ddfd              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4        [54, 510]   
0020ccbbb6d84e358d3414a3ff76cffd 2298d6c36e964ae4a3e7e9706d1fb8c2            [222]   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [600]   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [378]   
003d66b6608740288d6cc97a6903f4f0 0b1e1539f2cc45b7b9fa7c272da2e1d7            [696]   
                                 3f207df678b143eea3cee63160fa8bed              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4       [384, 504]   
00426fe3ffde4c6b9cb9ad6d077a13ea 0b1e1539f2cc45b7b9fa7c272da2e1d7              NaN   
                                 2906b810c7d4411798c6938adc9daaa5              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4            [258]   
004b041fbfe44859945daa2c7f79ee64 3f207df678b143eea3cee63160fa8bed              NaN   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [534]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [714]   
...                                                                            ...   
ffed75d3abc64b488982f50ed12878b5 4d5c57ea9a6940dd891ad53e9dbe8da0              NaN   
                                 ae264e3637204a6fb9bb56bc8210ddfd              NaN   
ffede3b700ac41d6a266fa1ba74b4f16 0b1e1539f2cc45b7b9fa7c272da2e1d7              NaN   
                                 4d5c57ea9a6940dd891ad53e9dbe8da0              NaN   
                                 f19421c1d4aa40978ebb69ca19b0e20d              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4              NaN   
fff0f0aac6c547b9b263080f09a5586a 3f207df678b143eea3cee63160fa8bed              NaN   
                                 4d5c57ea9a6940dd891ad53e9dbe8da0              NaN   
fff29fb549084123bd046dbc5ceb4faa 4d5c57ea9a6940dd891ad53e9dbe8da0       [426, 516]   
                                 ae264e3637204a6fb9bb56bc8210ddfd       [168, 684]   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [426]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [168]   
fff3ba4757bd42088c044ca26d73817a 2906b810c7d4411798c6938adc9daaa5            [528]   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [168]   
                         

In [28]:
def check_views_completions(customer_offer_pair):
    #No view or completion should be yes?
    if customer_offer_pair[1]['offer completed'] == np.nan:
        return (0)
    elif customer_offer_pair[1]['offer viewed'] == np.nan:
        return (0)
    elif len(customer_offer_pair[1]['offer completed']) > len(customer_offer_pair[1]['offer viewed']):
        return (0)
    else:
        return(1)

In [22]:
def value_calculation(transaction_amount, customer_id, reward):
    baseline = np.mean(unoffered_dict[customer_id])
    spending_gain = transaction_amount - baseline
    #https://ycharts.com/companies/SBUX/gross_profit_margin
    gain_cost = (spending_gain*.7221) + reward
    profitability = spending_gain - gain_cost
    return (profitability)


In [66]:
def make_decision(master_list=master_list):
    decision = []
    for customer_offer_pair in master_list.iterrows():
        customer_id = customer_offer_pair[0][0]
        reward_amt = customer_offer_pair[1]['reward']
        if check_views_completions == 0:
            decision.append('No')
        elif value_calculation(np.mean(customer_offer_pair[1]['trans_amount']), customer_id, reward_amt) < 0:
            decision.append('No')
        #elif not completed
        else:
            decision.append('Yes')
    return(decision)
master_list['decision']  =make_decision(master_list)
#len(master_list)

/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [67]:
master_list

offer completed  \
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5            [576]   
                                 3f207df678b143eea3cee63160fa8bed              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [414]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [528]   
00116118485d4dfda04fdbaba9a87b5c f19421c1d4aa40978ebb69ca19b0e20d              NaN   
0011e0d4e6b944f998e987f904e8c1e5 0b1e1539f2cc45b7b9fa7c272da2e1d7            [576]   
                                 2298d6c36e964ae4a3e7e9706d1fb8c2            [252]   
                                 3f207df678b143eea3cee63160fa8bed              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [576]   
0020c2b971eb4e9188eac86d93036a77 4d5c57ea9a6940dd891ad53e9dbe8da0            [510]   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 ae264e3637204a6fb9bb56bc8210ddfd              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4        [54, 510]   
0020ccbbb6d84e358d3414a3ff76cffd 2298d6c36e964ae4a3e7e9706d1fb8c2            [222]   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [600]   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [378]   
003d66b6608740288d6cc97a6903f4f0 0b1e1539f2cc45b7b9fa7c272da2e1d7            [696]   
                                 3f207df678b143eea3cee63160fa8bed              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4       [384, 504]   
00426fe3ffde4c6b9cb9ad6d077a13ea 0b1e1539f2cc45b7b9fa7c272da2e1d7              NaN   
                                 2906b810c7d4411798c6938adc9daaa5              NaN   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4            [258]   
004b041fbfe44859945daa2c7f79ee64 3f207df678b143eea3cee63160fa8bed              NaN   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [534]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [714]   
...                                                                            ...   
ffed75d3abc64b488982f50ed12878b5 4d5c57ea9a6940dd891ad53e9dbe8da0              NaN   
                                 ae264e3637204a6fb9bb56bc8210ddfd              NaN   
ffede3b700ac41d6a266fa1ba74b4f16 0b1e1539f2cc45b7b9fa7c272da2e1d7              NaN   
                                 4d5c57ea9a6940dd891ad53e9dbe8da0              NaN   
                                 f19421c1d4aa40978ebb69ca19b0e20d              NaN   
                                 fafdcd668e3743c1bb461111dcafc2a4              NaN   
fff0f0aac6c547b9b263080f09a5586a 3f207df678b143eea3cee63160fa8bed              NaN   
                                 4d5c57ea9a6940dd891ad53e9dbe8da0              NaN   
fff29fb549084123bd046dbc5ceb4faa 4d5c57ea9a6940dd891ad53e9dbe8da0       [426, 516]   
                                 ae264e3637204a6fb9bb56bc8210ddfd       [168, 684]   
                                 f19421c1d4aa40978ebb69ca19b0e20d            [426]   
                                 fafdcd668e3743c1bb461111dcafc2a4            [168]   
fff3ba4757bd42088c044ca26d73817a 2906b810c7d4411798c6938adc9daaa5            [528]   
                                 5a8bc65990b245e5a138643cd4eb9837              NaN   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9            [168]   
                         

In [74]:
master_list.to_csv('master_list_save')

In [2]:
master = pd.read_csv('master_list_save')

In [3]:
master


,Unnamed: 0,Unnamed: 1,offer completed,offer received,offer viewed,trans_amount,reward,decision
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,[576],[576],NaN,[10.27],2,No
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,NaN,[336],[372],NaN,0,Yes
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,NaN,[168],[192],NaN,0,Yes
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,[414],[408],[456],[8.57],5,No
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,[528],[504],[540],[14.11],2,No
5,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,NaN,"[168, 576]","[216, 630]",NaN,5,Yes
6,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,[576],[408],[432],[22.05],5,No
7,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,[252],[168],[186],[11.93],3,No
8,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,NaN,[0],[6],NaN,0,Yes
9,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,NaN,[336],[354],NaN,0,Yes


In [4]:
offer_one = master[master['Unnamed: 1'] == 'fafdcd668e3743c1bb461111dcafc2a4']
from sklearn.cross_validation import train_test_split




X =  pd.DataFrame(offer_one['Unnamed: 0']).merge(profile, left_on = 'Unnamed: 0', right_on = 'id')
X.set_index('id', inplace = True)
X.drop('Unnamed: 0', axis = 1, inplace = True)
X['gender'].replace('F', 1, inplace = True)
X['gender'].replace('M', 0, inplace = True)
X['gender'].replace('O', 2, inplace = True)
X['gender'].fillna(-1, inplace =True)
X['income'].fillna(method = 'ffill', inplace =True)
#
y = offer_one['decision']

X_train, X_test, y_train, y_test = train_test_split(X,y)

X

,age,became_member_on,gender,income
id,,,,
0009655768c64bdeb2e877511632db8f,33,20170421,0.0,72000.0
0020c2b971eb4e9188eac86d93036a77,59,20160304,1.0,90000.0
003d66b6608740288d6cc97a6903f4f0,26,20170621,1.0,73000.0
00426fe3ffde4c6b9cb9ad6d077a13ea,19,20160809,1.0,65000.0
004b041fbfe44859945daa2c7f79ee64,55,20180508,1.0,74000.0
004c5799adbf42868b9cff0396190900,54,20160331,0.0,99000.0
00840a2ca5d2408e982d56544dc14ffd,26,20141221,0.0,61000.0
0099bf30e4cb4265875266eb3eb25eab,61,20180214,0.0,66000.0
00b18b535d6d4f779dea4dc9ac451478,72,20170407,0.0,102000.0


In [5]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

array(['No', 'Yes', 'No', ..., 'No', 'No', 'No'], dtype=object)

In [9]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, preds)

0.66013897662665821

In [8]:
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=2)
ftwo_scorer()

TypeError: __call__() missing 3 required positional arguments: 'estimator', 'X', and 'y_true'

<H1> To do</H1>

-Improve timing of make_completion_rows  
    -Save offer_ids for information only  
    -Unpack dictionary before function  
-run make_completion_rows and create_unoffered_list under same for loop  
-Save unoffered list to dict(DONE) 
-Find mean, max, and mode(MEAN ONLY) 
-Compare Transaction to them(MEAN ONLY) 
-Threshold amount/reward amount(REWARD ONLY)
-calculate: (offered -unoffered - reward/unoffered) - margin(USED DIFFERENT CALCUALATION)
-make y/n on customer/offer pairs(DONE) 
-Machine learning.

<H1>Edge Cases</H1>

-Offer sent more than once  
-Transaction completes more than one offer

Future builds
-Transaction Frequency